In [8]:
from matplotlib import pyplot as plt
import numpy as np
import os.path
shape_size = 48
freq = np.genfromtxt('data/freq.txt')
def spec(batch, ran):
    path = 'data/DATA'+str(batch)+'/DATA'+str(batch)+'_sp'
    miss = []
    for i in ran:
        if not os.path.exists(path+str(i)+'.txt'):
            miss.append(i)
    return miss

import meep as mp
import math
import cmath

shape_size = 48
sx, sy, sz = 1, 1, 4
h = 1.25
dpml = 0.5
b_m, c_m = 1.4, 3.54
res = 25
echo = 1000
cell_size = mp.Vector3(sx,sy,sz)
fcen = 0.5
df = 0.2
theta = math.radians(0)
nfreq = 200

# k with correct length (plane of incidence: XZ) 
k = mp.Vector3(math.sin(theta),0,math.cos(theta)).scale(fcen)
def pw_amp(k, x0):
    def _pw_amp(x):
        return cmath.exp(1j * 2 * math.pi * k.dot(x + x0))
    return _pw_amp

def get_spec(batch, index):
    b = np.genfromtxt('data/DATA'+str(batch) +'_sh.txt')
    start = index * shape_size
    xc = b[start:start + shape_size, 0]
    yc = b[start:start + shape_size, 1]
    vertices = [mp.Vector3(xc[0],yc[0])]

    for j in range(1, len(xc) - 1):
        # eliminate duplicate point
        if xc[j] == xc[j - 1] and yc[j] == yc[j - 1]:
            continue
        vertices.append(mp.Vector3(xc[j], yc[j]))
        #print(xc[i][j], yc[i][j])
    # calculate transmission
    bend = get_bend(vertices)
    Ts = []
    st = np.genfromtxt('data/straight.txt')
    for j in range(nfreq):
        Ts = np.append(Ts, bend[j]/st[j])
    return Ts
#     plt.axis('equal')
#     plt.ylim(-0.5, 0.5)
#     plt.xlim(-0.5, 0.5)
#     plt.plot(x, y)
#     plt.fill(x, y)
#     plt.show()
def get_bend(vertices):
    geometry = [mp.Block(size = cell_size, material=mp.Medium(index=b_m)),
                mp.Prism(vertices, 
                         height=h, 
                         material=mp.Medium(index=c_m),
                         center=mp.Vector3()
                        )]
    pml_layers = [mp.PML(thickness=1, direction = mp.Z, side=mp.High),
                  mp.Absorber(thickness=1,direction = mp.Z, side=mp.Low)]
    src_pos = -(sz/2 - dpml - 0.5)
    src = [mp.Source(src = mp.GaussianSource(fcen, fwidth=df),
                     component = mp.Ey,
                     center = mp.Vector3(0,0,src_pos),
                     size = mp.Vector3(sx,sy,0),
                     amp_func=pw_amp(k,mp.Vector3(0,0,src_pos)))]
    sim = mp.Simulation(resolution=res,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        sources=src,
                        geometry=geometry,
                        k_point=k)
    freg = mp.FluxRegion(center=mp.Vector3(0,0,-src_pos),
                         size = mp.Vector3(sx,sy,0))
    trans = sim.add_flux(fcen, df, nfreq, freg)
    sim.run(until = echo)
    bend = mp.get_fluxes(trans)
    return bend

In [ ]:
batch = 46
ran = [i for i in range(100)]
miss = spec(batch, ran)
freq = np.genfromtxt('data/freq.txt')
for i in ran:
    
    path = 'data/DATA'+str(batch)+'/DATA'+str(batch)+'_sp'
    if i in miss:
        print('Batch: '+str(batch)+ ' sample: '+str(i))
        Ts = get_spec(batch,i)
        np.savetxt(path +str(i)+'.txt', Ts)
        Ts = np.genfromtxt(path +str(i)+'.txt')
        plt.ylim(0, 1.1)
        plt.plot(freq, Ts)
        plt.show()

Batch: 46 sample: 80
-----------
Initializing structure...
Meep: using complex fields.
Meep progress: 3.74/1000.0 = 0.4% done in 4.0s, 1069.3s to go
Meep progress: 7.78/1000.0 = 0.8% done in 8.0s, 1023.8s to go
Meep progress: 11.78/1000.0 = 1.2% done in 12.0s, 1010.6s to go
Meep progress: 15.64/1000.0 = 1.6% done in 16.1s, 1010.6s to go
Meep progress: 19.66/1000.0 = 2.0% done in 20.1s, 1000.2s to go
Meep progress: 23.52/1000.0 = 2.4% done in 24.1s, 999.4s to go
Meep progress: 27.54/1000.0 = 2.8% done in 28.1s, 991.6s to go
Meep progress: 31.62/1000.0 = 3.2% done in 32.1s, 982.8s to go
Meep progress: 35.68/1000.0 = 3.6% done in 36.1s, 975.5s to go
Meep progress: 39.76/1000.0 = 4.0% done in 40.1s, 968.6s to go
Meep progress: 43.96/1000.0 = 4.4% done in 44.1s, 959.4s to go
Meep progress: 48.04/1000.0 = 4.8% done in 48.1s, 953.5s to go
Meep progress: 52.22/1000.0 = 5.2% done in 52.1s, 945.9s to go
Meep progress: 56.42/1000.0 = 5.6% done in 56.1s, 938.8s to go
Meep progress: 60.660000000000

Meep progress: 495.42/1000.0 = 49.5% done in 489.2s, 498.3s to go
Meep progress: 499.32/1000.0 = 49.9% done in 493.2s, 494.6s to go
Meep progress: 503.28000000000003/1000.0 = 50.3% done in 497.3s, 490.8s to go
Meep progress: 507.2/1000.0 = 50.7% done in 501.3s, 487.0s to go
Meep progress: 511.06/1000.0 = 51.1% done in 505.3s, 483.4s to go
Meep progress: 515.1800000000001/1000.0 = 51.5% done in 509.3s, 479.3s to go
Meep progress: 519.14/1000.0 = 51.9% done in 513.3s, 475.4s to go
Meep progress: 523.24/1000.0 = 52.3% done in 517.3s, 471.4s to go
Meep progress: 527.14/1000.0 = 52.7% done in 521.3s, 467.6s to go
Meep progress: 531.04/1000.0 = 53.1% done in 525.3s, 463.9s to go
Meep progress: 535.16/1000.0 = 53.5% done in 529.3s, 459.8s to go
Meep progress: 539.38/1000.0 = 53.9% done in 533.3s, 455.5s to go
Meep progress: 543.52/1000.0 = 54.4% done in 537.4s, 451.3s to go
Meep progress: 547.6/1000.0 = 54.8% done in 541.4s, 447.2s to go
Meep progress: 551.58/1000.0 = 55.2% done in 545.4s, 44